In [1]:
version='knn_comps'

'''
knn baseline for comps
'''

'\nknn baseline for comps\n'

In [2]:

import collections
import copy
import csv
import math
import numbers
import random
import threading
import time
import types
import warnings
from functools import partial, update_wrapper

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import mean_absolute_error, median_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader, NeighborLoader, NeighborSampler
from torch_geometric.nn import SAGEConv, TransformerConv,NNConv
from torch.utils.data import DataLoader, IterableDataset
from tqdm import tqdm
import torch_geometric
torch_geometric.__version__
torch.__version__
print(torch.version.cuda)
import sys
# sys.path
# sys.path.append('zestimate-neural-net')
# from pyspark import SparkContext, SparkConf
# from pyspark.sql import DataFrame, SparkSession
# from pyspark.sql import functions as F
# from pyspark.sql import types as T
# from pyspark.sql.window import Window

# from aip_spark_sdk import AwsHelper
import pandas as pd
import numpy as np
import collections
import copy
import csv
import math
# import numbers
# import random
# import threading
# import time
# import types
import warnings
from functools import partial, update_wrapper

import matplotlib.pyplot as plt
# import networkx as nx
import numpy as np
import pandas as pd
# import torch.nn as nn
# import torch.nn.functional as F
# from sklearn.metrics import mean_absolute_error, median_absolute_error
# from sklearn.metrics import mean_absolute_percentage_error
# from sklearn.metrics import r2_score
from sklearn.neighbors import BallTree
pd.set_option('display.max_columns', None)
print(torch.cuda.is_available())

11.6
True


In [3]:
def get_feature_dict():
    num_features = [
        "latitude",
        "longitude",
        "year_built",
        "year_reno",
        "grade",
        "fbsmt_grade",
        "condition",
        "stories",
        "beds",
        "bath_full",
        "bath_3qtr",
        "bath_half",
    ]
    num_log_features = [
        "land_val",
        "imp_val",
        "sqft_lot",
        "sqft",
        "sqft_1",
        "sqft_fbsmt",
        "garb_sqft",
        "gara_sqft",
    ]
    cat_features = [
        # "area",
        "city",
       "submarket",
        "zoning",
        # "present_use",

        # "wfnt",
        # "golf",
        # "greenbelt",
        # "noise_traffic",
        # "view_rainier",
        # "view_olympics",
        # "view_cascades",
        # "view_territorial",
        # "view_skyline",
        # "view_sound",
        # "view_lakewash",
        # "view_lakesamm",
        # "view_otherwater",
        # "view_other",
        
        # "sale_date_yyyymm",
        # "sale_decade",
        # "sale_year",
        # "sale_week",
    ]
    ord_features = [
        # "stories",
        # "beds",
        # "bath_full",
        # "bath_3qtr",
        # "bath_half",
    ]
    time_features = ["sale_date",]

    feature_dict = {
        "nums": num_features,
        "num_logs": num_log_features,
        "cats": cat_features,
        "ords": ord_features,
        "time": time_features,
    }

    return feature_dict


feature_dict = get_feature_dict()

id_cols = ["sale_id", "pinx", "submarket"]
response_col = "sale_price"
feature_cols = set(
    feature_dict["nums"] +
    feature_dict["num_logs"] +
    feature_dict["cats"] +
    feature_dict["ords"] +
    feature_dict["time"]
)

In [4]:
class ResponseTransformer:
    """Response transformer."""
    def __init__(self):
        self.median = None

    def fit(self, y):
        y_trans = np.log1p(y)
        self.median = np.median(y_trans)
        return self

    def transform(self, y):
        y = np.log1p(y) - self.median
        return y

    def fit_transform(self, y):
        return self.fit(y).transform(y)

    def inverse_transform(self, y):
        y = np.expm1(y + self.median)
        return y

In [5]:
df=pd.read_csv('kingcountysales.csv')
df["sale_date"]=pd.to_datetime(df["sale_date"])
max_train_date = "2021-01-01"
min_train_date = "2016-01-01"
sample_beg_date= "2015-01-01"
df_1= df[df["sale_date"] >= min_train_date].copy()


In [6]:
def prepare_missing(df):
    df = (
        df
        .assign(imp_val=lambda x: x['imp_val'].replace(0, np.nan))
        .assign(land_val=lambda x: x['land_val'].replace(0, np.nan))
        .assign(sqft=lambda x: x['sqft'].replace(0, np.nan))
        .assign(beds=lambda x: x['beds'].replace(54, np.nan))
    )
    return df
def clean_sqft(df):
    sqft_1 = df.query('sqft < sqft_1')['sqft_1']
    sqft_fbsmt = df.query('sqft < sqft_1')['sqft_fbsmt']
    df.loc[df['sqft'] < df['sqft_1'], 'sqft'] = sqft_1 + sqft_fbsmt
    return df

def clean_year_built(df):
    df['build_type'] = 'Standard'
    df.loc[(df['year_built'] - 1) == pd.to_datetime(df['sale_date']).dt.year, 'build_type'] = 'New Construction'
    df.loc[(df['year_built'] - 1) > pd.to_datetime(df['sale_date']).dt.year, 'build_type'] = 'Lot Sale'
    return df

def prepare_outliers(df):
    df = (
        df
        .pipe(clean_sqft)
        .pipe(clean_year_built)
    )
    return df

def preprocessor(df_1, feature_c):
    df_out=df_1.copy()
    # feature_cols=feature_c.copy()
    
    '''
    categorical features: replace each category with its price statistics from prior years  
    '''
    df_sample=  df[(df["sale_date"] < min_train_date) & (df["sale_date"] >= sample_beg_date)].copy()
    trans_1=ResponseTransformer()
    df_sample[response_col]=trans_1.fit_transform(df_sample[response_col].values)
    # df_sample[response_col]= np.log1p(df_sample[response_col].values) - df_sample[response_col].median()
    for cat_col in feature_dict['cats']:
        cat_stats= df_sample[[cat_col,response_col]].groupby(cat_col).describe().add_prefix(f'{cat_col}_')[f'{cat_col}_sale_price'].reset_index()
        df_out=df_out.merge(cat_stats, how='left', on=cat_col)
        print(cat_col)
        print(df_out.shape)
        print(df_out['sale_date'].isna().sum())
        df_out.drop(columns=cat_col, inplace=True)
        feature_cols.update(list(cat_stats.columns))
        feature_cols.remove(cat_col)
        
    '''
    temporal adjusting 
    '''
    print(df_out['sale_date'].isna().sum())
    date_time = df_out['sale_date']

    timestamp_s = date_time.dt.week
    # df_1['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
    # df_1['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
    df_out['year_sin'] = np.sin(timestamp_s * (2 * np.pi / 52))
    df_out['year_cos'] = np.cos(timestamp_s * (2 * np.pi / 52))
    # df_out.drop(columns='sale_date', inplace=True)
    df_out['trans_year']=date_time.dt.year
    feature_cols.update(['trans_year','year_sin','year_cos'])
    feature_cols.remove('sale_date')
    
    '''
    taking log
    '''
    
    df_out[feature_dict['num_logs']]=np.log1p(df_out[feature_dict['num_logs']])
    '''
    filling missing values
    '''
    
    df_out.fillna(df_out.median(), inplace=True)
    # print(df_out.isna().sum())
    # df_out.dropna(inplace=True)
    return df_out, feature_cols
        

In [7]:
df_1.shape

(161512, 47)

In [8]:
old_feat_col=feature_cols.copy()
df_1=prepare_missing(df_1)
df_1=prepare_outliers(df_1)


# df_1=df_1.loc[df_1['city']=='SEATTLE']


df_out,feature_cols_2=preprocessor(df_1, feature_cols)
df_out["sale_year"] = df_out["sale_date"].dt.year
df_out["sale_month"] = df_out["sale_date"].dt.month
df_out["sale_week"] = df_out["sale_date"].dt.isocalendar().week
df_out["sale_day"] = df_out["sale_date"].dt.day

city
(161512, 56)
0
submarket
(161512, 63)
0
zoning
(161512, 70)
0
0


/tmp/ipykernel_132/1722977719.py:57: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  timestamp_s = date_time.dt.week
/tmp/ipykernel_132/1722977719.py:76: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  df_out.fillna(df_out.median(), inplace=True)
/tmp/ipykernel_132/1722977719.py:76: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_out.fillna(df_out.median(), inplace=True)


In [9]:
df_train = df_out[(df_out["sale_date"] < max_train_date) & (df_out["sale_date"] >= min_train_date)].copy()
df_train = df_train.reset_index(drop=True)
df_test = df_out[df_out["sale_date"] >= max_train_date].copy()
df_test = df_test.reset_index(drop=True)



response_transformer = ResponseTransformer()




X_train=df_train[feature_cols].copy()
y_train=response_transformer.fit_transform(df_train[response_col])

X_test=df_test[feature_cols].copy()
y_test=df_test[response_col].copy()



from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X_train[list(X_train.columns)]=scaler.fit_transform(X_train)
X_test[list(X_test.columns)]=scaler.transform(X_test)




/tmp/ipykernel_132/3924654007.py:13: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X_train=df_train[feature_cols].copy()
/tmp/ipykernel_132/3924654007.py:16: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  X_test=df_test[feature_cols].copy()


In [10]:
def mean_percentage_error(true, pred, epsilon=1e-6):
    """Mean Percentage Error (MPE)."""
    pe = (pred - true) / (true + epsilon)
    return np.mean(pe)


def median_absolute_percentage_error(true, pred, epsilon=1e-6):
    """Median Absolute Percentage Error (MdAPE)."""
    ape = np.abs(pred - true) / (true + epsilon)
    return np.median(ape)


def median_percentage_error(true, pred, epsilon=1e-6):
    """Median Percentage Error (MdPE)."""
    pe = (pred - true) / (true + epsilon)
    return np.median(pe)


def frac_within_ci(true, pred_low, pred_high):
    """Fraction within Confidence Interval."""
    wci = (true >= pred_low) & (true <= pred_high)
    return np.mean(wci)


def mean_ci_width(true, pred_low, pred_high, epsilon=1e-6):
    """Mean Confidence Interval Width."""
    ciw = (pred_high - pred_low) / (true + epsilon)
    return np.mean(ciw)


def median_ci_width(true, pred_low, pred_high, epsilon=1e-6):
    """Median Confidence Interval Width."""
    ciw = (pred_high - pred_low) / (true + epsilon)
    return np.median(ciw)


def evaluate_preds(y_test, y_pred, y_pred_low=None, y_pred_high=None):
    output = {}
    output["Count"] = len(y_test)
    output["R2"] = r2_score(y_test, y_pred)
    output["MAE"] = mean_absolute_error(y_test, y_pred)
    output["MdAE"] = median_absolute_error(y_test, y_pred)
    output["MPE"] = mean_percentage_error(y_test, y_pred)
    output["MdPE"] = median_percentage_error(y_test, y_pred)
    output["MAPE"] = mean_absolute_percentage_error(y_test, y_pred)
    output["MdAPE"] = median_absolute_percentage_error(y_test, y_pred)
    if y_pred_low is not None and y_pred_high is not None:
        output["Pct. Within CI"] = frac_within_ci(y_test, y_pred_low, y_pred_high)
        output["Mean CI Width"] = mean_ci_width(y_test, y_pred_low, y_pred_high)
        output["Med. CI Width"] = median_ci_width(y_test, y_pred_low, y_pred_high)
    return output

In [11]:
'''
mostly referencing Reid's public notebook on kaggle for typical preprocessing steps 
https://www.kaggle.com/code/reidjohnson/house-price-eda-and-modeling-with-python

'''

"\nmostly referencing Reid's public notebook on kaggle for typical preprocessing steps \nhttps://www.kaggle.com/code/reidjohnson/house-price-eda-and-modeling-with-python\n\n"

In [12]:
X_train=X_train[['latitude', 'longitude']]
X_test=X_test[['latitude', 'longitude']]
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train.values, y_train.values)
y_pred_knn=knn.predict(X_test.values)

In [13]:
y_pred_knn= response_transformer.inverse_transform(y_pred_knn)
evaluate_preds(y_test.values, y_pred_knn)
pd.DataFrame([evaluate_preds(y_test, y_pred_knn)], index=[f"{version}"])

,Count,R2,MAE,MdAE,MPE,MdPE,MAPE,MdAPE
knn_comps,30995,0.410641,338154.07411,214865.082271,-0.224494,-0.269288,0.288875,0.282975
